In [ ]:
#|default_exp showdoc

# showdoc
> Display symbol documentation in notebook and website

In [ ]:
#|export
from fastcore.docments import *
from fastcore.utils import *
from importlib import import_module
import inspect

from nbprocess.read import get_config

In [ ]:
#|export
def get_name(obj):
    "Get the name of `obj`"
    if hasattr(obj, '__name__'):       return obj.__name__
    elif getattr(obj, '_name', False): return obj._name
    elif hasattr(obj,'__origin__'):    return str(obj.__origin__).split('.')[-1] #for types
    elif type(obj)==property:          return _get_property_name(obj)
    else:                              return str(obj).split('.')[-1]

In [ ]:
#|export
def qual_name(obj):
    "Get the qualified name of `obj`"
    if hasattr(obj,'__qualname__'): return obj.__qualname__
    if inspect.ismethod(obj):       return f"{get_name(obj.__self__)}.{get_name(fn)}"
    return get_name(obj)

In [ ]:
#|export
class ShowDocRenderer:
    def __init__(self, sym, disp:bool=True):
        "Show documentation for `sym`"
        store_attr()
        self.nm = qual_name(sym)
        self.isfunc = inspect.isfunction(sym)
        self.sig = inspect.signature(sym)
        self.docs = docstring(sym)

In [ ]:
#|export
class BasicMarkdownRenderer(ShowDocRenderer):
    def _repr_markdown_(self):
        doc = '---\n\n'
        if self.isfunc: doc += '#'
        doc += f'### {self.nm}\n\n> **`{self.nm}`**` {self.sig}`'
        if self.docs: doc += f"\n\n{self.docs}"
        return doc

In [ ]:
#|export
def show_doc(sym, disp=True, renderer=None):
    if renderer is None: renderer = get_config().get('renderer', None)
    if renderer is None: renderer=BasicMarkdownRenderer
    elif isinstance(renderer,str):
        p,m = renderer.rsplit('.', 1)
        renderer = getattr(import_module(p), m)
    return renderer(sym or show_doc, disp=disp)

You can use `show_doc` to document apis of functions, classes or methods:

In [ ]:
def f(x:int=1):
    "func docstring"
    ...

show_doc(f)

---

#### f

> **`f`**` (x: int = 1)`

func docstring

In [ ]:
class Foo:
    def __init__(d:str,e:int):
        "This is the docstring for the __init__ method"
        ...

show_doc(Foo)

---

### Foo

> **`Foo`**` (e: int)`

This is the docstring for the __init__ method

In [ ]:
class Foo:
    def a_method(a:list,b:dict,c):
        "This is a method"
        ...

show_doc(Foo.a_method)

---

#### Foo.a_method

> **`Foo.a_method`**` (a: list, b: dict, c)`

This is a method

In [ ]:
#|export
class BasicHtmlRenderer(ShowDocRenderer):
    def _repr_html_(self):
        doc = '<hr/>\n'
        lvl = 4 if self.isfunc else 3
        doc += f'<h{lvl}>{self.nm}</h{lvl}>\n<blockquote><code>{self.nm}{self.sig}</code></blockquote>'
        if self.docs: doc += f"<p>{self.docs}</p>"
        return doc

In [ ]:
class F:
    "class docstring"
    def __init__(self, x:int=1): ...

show_doc(F, renderer=BasicHtmlRenderer)

In [ ]:
#|export
def _is_patch(tree):
    decs = [d.id for d in getattr(tree, 'decorator_list', []) if hasattr(d, 'id')]
    return 'patch' in decs

In [ ]:
#|export
def get_patch_name(tree):
    "If a method is defined with @patch, get fully qualified name."
    first_arg = first(L(nested_attr(tree, 'args.args')))
    if _is_patch(tree) and first_arg.arg == 'self':
        annotation = nested_attr(first_arg, 'annotation.id')
        if annotation: return f'{annotation}.{tree.name}'
        else: return tree.name
    else: return tree.name

In [ ]:
#|hide
import ast
code="""
@bar
@patch
@foo
def a_method(self:Foo, a:list,b:dict,c):
    "This is a method"
    ...
"""

code2="""
@bar
@foo
def a_method(self:Foo, a:list,b:dict,c):
    "This is a method"
    ...
"""

_tree = ast.parse(code).body[0]
assert _is_patch(_tree)
test_eq(get_patch_name(_tree), 'Foo.a_method')

_tree2 = ast.parse(code2).body[0]
test_eq(get_patch_name(_tree2), 'a_method')

## Export -

In [ ]:
#|hide
#|eval: false
from nbprocess.doclinks import nbprocess_export
nbprocess_export()